In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
linkTrain = 'https://drive.google.com/file/d/1uV8qsVnl1XLxLlfw2tnGF3rbIU3tntHI/view?usp=sharing'
linkIdTrain = '1uV8qsVnl1XLxLlfw2tnGF3rbIU3tntHI'

linkTest = 'https://drive.google.com/file/d/1QIJWtK99SblVQK88AHqHwAVBZJcT14yW/view?usp=sharing'
LinkIdTest = '1QIJWtK99SblVQK88AHqHwAVBZJcT14yW'

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
downloaded = drive.CreateFile({'id':linkIdTrain}) 
downloaded.GetContentFile('Bot_IOT_Train.csv')  

downloaded = drive.CreateFile({'id':LinkIdTest}) 
downloaded.GetContentFile('Bot_IOT_Test.csv')  

#loading data
df_GB_Train = pd.read_csv('Bot_IOT_Train.csv')
df_GB_Test = pd.read_csv('Bot_IOT_Test.csv')

In [ ]:
#Data preprocessing
#checking for NULL Values for training
null_columns = df_GB_Train.columns[df_GB_Train.isnull().any()]
df_GB_Train[null_columns].isnull().sum()

#for testing

null_columns = df_GB_Test.columns[df_GB_Test.isnull().any()]
df_GB_Test[null_columns].isnull().sum()

Series([], dtype: float64)

In [ ]:
#ignoring non important attributes from dataset
#training
df_GB_Train=df_GB_Train.drop(['pkSeqID', 'saddr', 'sport', 'daddr', 'dport', 'proto'], axis = 1)

#for testing
df_GB_Test=df_GB_Test.drop(['pkSeqID', 'saddr', 'sport', 'daddr', 'dport', 'proto'], axis = 1)

In [ ]:
# import machine learning algorithms
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
#dependent and independent variables
X_Train = df_GB_Train.iloc[:, 0:10]
Y_Train = df_GB_Train.loc[:, 'attack']


# from imblearn.over_sampling import SMOTE                        # Balance imbalance data
# sm = SMOTE(random_state=2)
# X_Train, Y_Train = sm.fit_sample(X_Train, Y_Train.ravel())
print(sum(Y_Train==0))
print(sum(Y_Train==1))

X_Test = df_GB_Test.iloc[: , 0:10]
Y_Test = df_GB_Test.loc[:,'attack']

print(X_Train.shape)
print(Y_Train.shape)
print(X_Test.shape)
print(Y_Test.shape)

370
2934447
(2934817, 10)
(2934817,)
(733705, 10)
(733705,)


In [ ]:

from sklearn.neighbors import KNeighborsClassifier
clf_KNN = KNeighborsClassifier(n_neighbors = 5)
clf_KNN.fit(X_Train, Y_Train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
#predicting
y_pred = clf_KNN.predict(X_Test)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
#model evaluation
accuracy = accuracy_score(Y_Test,y_pred)*100
accuracy

99.99291268289026

In [ ]:
#confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(Y_Test, y_pred))

Confusion Matrix:
[[    59     48]
 [     4 733594]]


In [ ]:
print("Classification Report")
print(classification_report(Y_Test, y_pred))

Classification Report
              precision    recall  f1-score   support

           0       0.94      0.55      0.69       107
           1       1.00      1.00      1.00    733598

    accuracy                           1.00    733705
   macro avg       0.97      0.78      0.85    733705
weighted avg       1.00      1.00      1.00    733705



In [ ]:
####################predicting category attribute
#encoding category and subcategory
from sklearn import preprocessing
lableEnc = preprocessing.LabelEncoder()

print(df_GB_Test['category'].unique())
print(df_GB_Train['category'].unique())

['DoS' 'DDoS' 'Reconnaissance' 'Normal' 'Theft']
['DDoS' 'DoS' 'Reconnaissance' 'Normal' 'Theft']


In [ ]:
#encoding 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le1 = LabelEncoder()
df_GB_Train['category']=le.fit_transform(df_GB_Train['category'])
df_GB_Train['subcategory']=le1.fit_transform(df_GB_Train['subcategory'])

df_GB_Test['category']=le.transform(df_GB_Test['category'])
df_GB_Test['subcategory']=le1.transform(df_GB_Test['subcategory'])

In [ ]:
print(df_GB_Test['category'].unique())
print(df_GB_Train['category'].unique())

[1 0 3 2 4]
[0 1 3 2 4]


In [ ]:
#dependent and independent variables
X_cat_train = df_GB_Train.iloc[:, 0:11]
Y_cat_train = df_GB_Train.loc[:, 'category']

X_cat_test =df_GB_Test.iloc[:, 0:10]
X_cat_test['attack']= y_pred
Y_cat_test=df_GB_Test.loc[:, 'category']

print(X_cat_train)
print(Y_cat_train)

            seq    stddev  N_IN_Conn_P_SrcIP  ...     srate       max  attack
0        251984  1.900363                100  ...  0.494549  4.031619       1
1        256724  0.078003                 38  ...  0.256493  4.012924       1
2         62921  0.268666                100  ...  0.294880  3.609205       1
3         99168  1.823185                 63  ...  0.461435  4.942302       1
4        105063  0.822418                100  ...  1.002999  4.994452       1
...         ...       ...                ...  ...       ...       ...     ...
2934812  253370  0.016992                100  ...  0.243473  4.124047       1
2934813  231693  1.922317                100  ...  0.490800  4.090534       1
2934814  158616  2.112228                100  ...  0.207444  4.332615       1
2934815  179855  0.000000                100  ...  0.162130  0.000000       1
2934816   95429  0.053820                100  ...  0.116388  0.172408       1

[2934817 rows x 11 columns]
0          0
1          0
2        

In [ ]:
#classifier
model_gb_cat = KNeighborsClassifier(n_neighbors = 5)
model_gb_cat.fit(X_cat_train, Y_cat_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
#calcuating accuracy of the model on test set
pred_cat = model_gb_cat.predict(X_cat_test)

#calculate accuracy
print("Accuracy Score: ")
print (accuracy_score(Y_cat_test, pred_cat)*100)

#confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(Y_cat_test, pred_cat))

#classification report
print("Classification Report")
print(classification_report(Y_cat_test, pred_cat))

Accuracy Score: 
89.45298178423208
Confusion Matrix:
[[348421  35724      0   1164      0]
 [ 37202 291762      1   1147      0]
 [     2     36     60      9      0]
 [   878   1217      3  16065      0]
 [     0      1      0      0     13]]
Classification Report
              precision    recall  f1-score   support

           0       0.90      0.90      0.90    385309
           1       0.89      0.88      0.89    330112
           2       0.94      0.56      0.70       107
           3       0.87      0.88      0.88     18163
           4       1.00      0.93      0.96        14

    accuracy                           0.89    733705
   macro avg       0.92      0.83      0.87    733705
weighted avg       0.89      0.89      0.89    733705



In [ ]:
####################Predicting subcategory of attack
#dependent and independent variables
X_subcat_train = df_GB_Train.iloc[:, 0:12]
Y_subcat_train = df_GB_Train.loc[:, 'subcategory']

X_subcat_test =df_GB_Test.iloc[:, 0:10]

X_subcat_test['attack']= y_pred
X_subcat_test['category']= pred_cat
Y_subcat_test=df_GB_Test.loc[:, 'subcategory']

In [ ]:
#classifying
model_gb_subcat = KNeighborsClassifier(n_neighbors=100)
model_gb_subcat.fit(X_subcat_train, Y_subcat_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=100, p=2,
                     weights='uniform')

In [ ]:
#calcuating accuracy of the model on test set
pred_subcat = model_gb_subcat.predict(X_subcat_test)

#calculate accuracy
print("Accuracy Score: ")
print (accuracy_score(Y_subcat_test, pred_subcat)*100)

#confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(Y_subcat_test, pred_subcat))

Accuracy Score: 
71.92863616848733
Confusion Matrix:
[[   157      0      1      0     83    133    130]
 [     0      5      0      0      0      9      0]
 [     0      0     23      3     49     15     17]
 [     0      0      0    514     63   2090    954]
 [    66      0      7     51   5047   5520   3851]
 [    44      0      4    172    968 181653 135496]
 [    33      0      3     85    773  55341 340345]]


In [ ]:
#df['attack', 'category', 'subcategory']
a=df_GB_Test['attack']
b=df_GB_Test['category']
c=df_GB_Test['subcategory']
count=0
for i in range(0,len(df_GB_Test)):
  if(y_pred[i]==a[i]) and (pred_cat[i]==b[i]):
    count=count+1
count
count/len(df_GB_Test)*100

89.45284548967228

In [ ]:
#df['attack', 'category', 'subcategory']
a=df_GB_Test['attack']
b=df_GB_Test['category']
c=df_GB_Test['subcategory']
count=0
for i in range(0,len(df_GB_Test)):
  if((y_pred[i]==a[i]) and (pred_cat[i]==b[i]) and (pred_cat[i]==c[i])):
    count=count+1
count
count/len(df_GB_Test)*100

0.038162476744740735